## Simple Customer Sentiment Analysis (NLP)
<b>using Tensorflow and Keras Index-based Encoding</b>

In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
sentences = [
    'Great service, the shipment is fast and the quality is good, thanks.',
    'There is a damage to the goods, i have to return the shipment',
    'The goods is exactly as ordered, fast respond seller, recommended',
    'Excellent, thank you',
    'Packaging broken, unfriendly seller, not recommended'
]

labels = [1,
          0,
          1,
          1,
          0]

### Keras Tokenizer (Index-based Encoding) and Padding Sequences

In [4]:
tokenizer = Tokenizer(num_words=1000, oov_token="<UNK>")

In [5]:
tokenizer.fit_on_texts(sentences)

In [6]:
word_index = tokenizer.word_index
word_index

{'<UNK>': 1,
 'the': 2,
 'is': 3,
 'shipment': 4,
 'fast': 5,
 'to': 6,
 'goods': 7,
 'seller': 8,
 'recommended': 9,
 'great': 10,
 'service': 11,
 'and': 12,
 'quality': 13,
 'good': 14,
 'thanks': 15,
 'there': 16,
 'a': 17,
 'damage': 18,
 'i': 19,
 'have': 20,
 'return': 21,
 'exactly': 22,
 'as': 23,
 'ordered': 24,
 'respond': 25,
 'excellent': 26,
 'thank': 27,
 'you': 28,
 'packaging': 29,
 'broken': 30,
 'unfriendly': 31,
 'not': 32}

In [7]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences

[[10, 11, 2, 4, 3, 5, 12, 2, 13, 3, 14, 15],
 [16, 3, 17, 18, 6, 2, 7, 19, 20, 6, 21, 2, 4],
 [2, 7, 3, 22, 23, 24, 5, 25, 8, 9],
 [26, 27, 28],
 [29, 30, 31, 8, 32, 9]]

In [8]:
padded = pad_sequences(sequences, padding='post', truncating='post')
padded

array([[10, 11,  2,  4,  3,  5, 12,  2, 13,  3, 14, 15,  0],
       [16,  3, 17, 18,  6,  2,  7, 19, 20,  6, 21,  2,  4],
       [ 2,  7,  3, 22, 23, 24,  5, 25,  8,  9,  0,  0,  0],
       [26, 27, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [29, 30, 31,  8, 32,  9,  0,  0,  0,  0,  0,  0,  0]])

In [9]:
pad_length = len(padded[0])
pad_length

13

In [10]:
labels = np.array(labels)
labels

array([1, 0, 1, 1, 0])

In [11]:
x = padded

### One-hot Encoding (for Labels)

In [12]:
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(labels.reshape(-1, 1))

In [13]:
y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

### Tensorflow Dense layers (Deep Neural Network layers) and Embedding Layers

In [14]:
vocab_size = 100
embedding_dim = 5

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=pad_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit(x, y, epochs=120)

Epoch 1/120
1/1 [==============================] - 1s 942ms/step - loss: 0.6851 - accuracy: 0.6000
Epoch 2/120
1/1 [==============================] - 0s 10ms/step - loss: 0.6776 - accuracy: 0.8000
Epoch 3/120
1/1 [==============================] - 0s 9ms/step - loss: 0.6705 - accuracy: 0.8000
Epoch 4/120
1/1 [==============================] - 0s 10ms/step - loss: 0.6637 - accuracy: 0.8000
Epoch 5/120
1/1 [==============================] - 0s 14ms/step - loss: 0.6567 - accuracy: 0.8000
Epoch 6/120
1/1 [==============================] - 0s 10ms/step - loss: 0.6493 - accuracy: 0.8000
Epoch 7/120
1/1 [==============================] - 0s 10ms/step - loss: 0.6417 - accuracy: 0.8000
Epoch 8/120
1/1 [==============================] - 0s 5ms/step - loss: 0.6339 - accuracy: 0.8000
Epoch 9/120
1/1 [==============================] - 0s 2ms/step - loss: 0.6260 - accuracy: 1.0000
Epoch 10/120
1/1 [==============================] - 0s 7ms/step - loss: 0.6178 - accuracy: 1.0000
Epoch 11/120
1/1 [====

### Test data

In [17]:
test_data = [
    'Bad quality, i must return the goods, not recommended',
    'Thanks, good quality product'
]

In [18]:
test_seq = tokenizer.texts_to_sequences(test_data)
test_seq

[[1, 13, 19, 1, 21, 2, 7, 32, 9], [15, 14, 13, 1]]

In [19]:
test_pad = pad_sequences(test_seq, padding='post', truncating='post', maxlen=pad_length)
test_pad

array([[ 1, 13, 19,  1, 21,  2,  7, 32,  9,  0,  0,  0,  0],
       [15, 14, 13,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [20]:
result = model.predict(test_pad)
result

array([[0.7144343 , 0.2855657 ],
       [0.1671744 , 0.83282554]], dtype=float32)

In [21]:
np.round(result)

array([[1., 0.],
       [0., 1.]], dtype=float32)

Label:<br>
[1, 0] = Negative<br>
[0, 1] = Positive<br>

In [26]:
print(f"Input:\n{test_data[0]}\n\tOutput: {np.argmax(np.round(result)[0])}\n")
print(f"Input:\n{test_data[1]}\n\tOutput: {np.argmax(np.round(result)[1])}\n")

Input:
Bad quality, i must return the goods, not recommended
	Output: 0

Input:
Thanks, good quality product
	Output: 1

